In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import sys

In [3]:
cif_names = ["7mhf", "7mhg", "7mhh", "7mhi", "7mhj", "7mhk"]
mapper_df = pd.read_csv(Path(Path.home(), "Documents/xray/dev/35_cif_combos/data/7mhf.csv"), index_col=0)
pdb_r_free_dict = {"7mhf": .224, "7mhg": .205, "7mhh": .253, "7mhi": .228, "7mhj": .240, "7mhk": .247}

exp_name = "189_exp_ref_10000"
analysis_dir = Path(Path.home(), "Documents/xray/sample_bench/data/7mhf", exp_name)
analysis_df = pd.read_csv(Path(analysis_dir, "sample.csv"), index_col=0)
analysis_df.head()

Ns = [1, 2, 4, 8, 16]
Js = list(range(1,7))

In [10]:
best_rows = list()

exp_deltas, ref_deltas, ref_ff_deltas, job_cif_strs = list(), list(), list(), list()
for N in Ns:
    for J in Js:
        for cif_name in cif_names:
            field = "r_free_{}".format(cif_name)

            N_J_cif_subset = analysis_df[(analysis_df["N"] == N) & (analysis_df["J"] == J)]
            best_row = N_J_cif_subset[N_J_cif_subset[field] == N_J_cif_subset[field].min()]
            best_rows.append(best_row)
            r_free = best_row[field].values[0]
            ff = best_row["ff"].values[0]
            exp_delta = r_free - pdb_r_free_dict[cif_name]
            exp_deltas.append(exp_delta)

            N1_J1_cif_subset = analysis_df[(analysis_df["N"] == 1) & (analysis_df["J"] == 1)]
            best_N1_J1_row = N1_J1_cif_subset[N1_J1_cif_subset[field] == N1_J1_cif_subset[field].min()]
            ref_r_free = best_N1_J1_row[field].values[0]
            ref_ff = best_N1_J1_row["ff"].values[0]
            ref_delta = r_free - ref_r_free
            ref_delta_ff = ff - ref_ff
            ref_deltas.append(ref_delta)
            ref_ff_deltas.append(ref_delta_ff)

            # job_id = best_row["job_id"].values[0]
            # job_cif_files = mapper_df.loc[job_id].values[0].split(",")
            # job_cif_names = [Path(job_cif_file).stem for job_cif_file in job_cif_files]
            # job_cif_str = ",".join(job_cif_names)
            # job_cif_strs.append(job_cif_str)

            # print(cif_name, N, J, job_id, r_free, delta, ff, pdb, job_cif_names)

summary_df = pd.concat(best_rows)
summary_df["exp_delta"] = exp_deltas
summary_df["ref_delta"] = ref_deltas
summary_df["ref_ff_delta"] = ref_ff_deltas
# summary_df["job_cif_str"] = job_cif_strs

summary_df.head()


,r_free_7mhf,ff,pdb,xray_7mhf,w_0_7mhf,N,J,job_id,r_free_7mhg,xray_7mhg,...,w_9_7mhk,w_10_7mhk,w_11_7mhk,w_12_7mhk,w_13_7mhk,w_14_7mhk,w_15_7mhk,exp_delta,ref_delta,ref_ff_delta
0,0.317513,551.709904,/wynton/group/sali/mhancock/xray/sample_bench/...,5.625852,1.0,1,1,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.093513,0.0,0.0
1,NaN,356.272686,/wynton/group/sali/mhancock/xray/sample_bench/...,NaN,NaN,1,1,1,0.297537,5.526696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.092537,0.0,0.0
2,NaN,910.272376,/wynton/group/sali/mhancock/xray/sample_bench/...,NaN,NaN,1,1,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049092,0.0,0.0
3,NaN,1039.888789,/wynton/group/sali/mhancock/xray/sample_bench/...,NaN,NaN,1,1,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.054967,0.0,0.0
4,NaN,442.542202,/wynton/group/sali/mhancock/xray/sample_bench/...,NaN,NaN,1,1,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.056949,0.0,0.0


In [11]:
summary_df.to_csv(analysis_dir, "summary.csv")

In [16]:
best_rows = list()
for cif_name in cif_names:
    field = "r_free_{}".format(cif_name)
    best_row = summary_df[summary_df[field] == summary_df[field].min()]
    print(best_row)
    best_rows.append(best_row)
    # best_row = cif_subset[cif_subset["exp_delta"] == cif_subset["exp_delta"].min()].iloc[0]
    # print(cif_name, best_row["N"], best_row["J"], best_row["exp_delta"], best_row["ref_delta"], best_row["pdb"])

summary_best_df = pd.concat(best_rows)
summary_best_df.head()

     r_free_7mhf           ff  \
198     0.279337  2424.053972   

                                                   pdb  xray_7mhf  w_0_7mhf  \
198  /wynton/group/sali/mhancock/xray/sample_bench/...   5.460668   0.49315   

     N  J  job_id  r_free_7mhg  xray_7mhg  ...  w_9_7mhk  w_10_7mhk  \
198  2  2      69          NaN        NaN  ...       NaN        NaN   

     w_11_7mhk  w_12_7mhk  w_13_7mhk  w_14_7mhk  w_15_7mhk  exp_delta  \
198        NaN        NaN        NaN        NaN        NaN   0.055337   

     ref_delta  ref_ff_delta  
198  -0.038176   1872.344068  

[1 rows x 116 columns]
     r_free_7mhf           ff  \
238          NaN  3459.900325   

                                                   pdb  xray_7mhf  w_0_7mhf  \
238  /wynton/group/sali/mhancock/xray/sample_bench/...        NaN       NaN   

     N  J  job_id  r_free_7mhg  xray_7mhg  ...  w_9_7mhk  w_10_7mhk  \
238  2  3      87     0.264894   5.337109  ...       NaN        NaN   

     w_11_7mhk  w_12_7mhk  w_

,r_free_7mhf,ff,pdb,xray_7mhf,w_0_7mhf,N,J,job_id,r_free_7mhg,xray_7mhg,...,w_9_7mhk,w_10_7mhk,w_11_7mhk,w_12_7mhk,w_13_7mhk,w_14_7mhk,w_15_7mhk,exp_delta,ref_delta,ref_ff_delta
198,0.279337,2424.053972,/wynton/group/sali/mhancock/xray/sample_bench/...,5.460668,0.49315,2,2,69,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.055337,-0.038176,1872.344068
238,NaN,3459.900325,/wynton/group/sali/mhancock/xray/sample_bench/...,NaN,NaN,2,3,87,0.264894,5.337109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.059894,-0.032643,3103.627639
728,NaN,42194.112853,/wynton/group/sali/mhancock/xray/sample_bench/...,NaN,NaN,8,4,244,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.055950,-0.105042,41283.840478
729,NaN,39311.353478,/wynton/group/sali/mhancock/xray/sample_bench/...,NaN,NaN,8,4,244,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.044297,-0.099265,38271.464689
854,NaN,67768.896208,/wynton/group/sali/mhancock/xray/sample_bench/...,NaN,NaN,16,3,289,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.047687,-0.104636,67326.354006


In [18]:
summary_best_df.to_csv(Path(analysis_dir, "summary_best.csv"))